# Getting Started with Azure Databricks

**Technical Accomplishments:**
- Set the stage for learning on the Databricks platform
- Create the cluster
- Discover the workspace, import table data
- Demonstrate how to develop & execute code within a notebook
- Review the various "Magic Commands"
- Introduce the Databricks File System (DBFS)

## Attach notebook to your cluster
Before executing any cells in the notebook, you need to attach it to your cluster. Make sure that the cluster is running.

In the notebook's toolbar, select the drop down arrow next to Detached, and then select your cluster under Attach to.

## Working with notebooks

A notebook is a web-based interface to a document that contains 
* runnable code
* visualizations
* descriptive text

To create a notebook, click on `Workspace`, browse into the desired folder, right click and choose `Create` then select `Notebook`.

A notebook contains multiple cells. Each cell has a specific type. 

A default programming language is configured when creating the notebook and it will be implicitly used for new cells.

#### Magic commands

We can override the cell's default programming language by using one of the following *magic commands* at the start of the cell:

* `%python` for cells running python code
* `%scala` for cells running scala code
* `%r` for cells running R code
* `%sql` for cells running sql code
  
Additional magic commands are available:

* `%md` for descriptive cells using markdown
* `%sh` for cells running shell commands
* `%run` for cells running code defined in a separate notebook
* `%fs` for cells running code that uses `dbutils` commands

To run a cell use one of the following options:
  * **CTRL+ENTER** or **CMD+RETURN**
  * **SHIFT+ENTER** or **SHIFT+RETURN** to run the cell and move to the next one
  * Using **Run Cell**, **Run All Above** or **Run All Below** as seen here<br/><img style="box-shadow: 5px 5px 5px 0px rgba(0,0,0,0.25); border: 1px solid rgba(0,0,0,0.25);" src="https://files.training.databricks.com/images/notebook-cell-run-cmd.png"/>

In [0]:
#assuming the default language for a notebook was set to Python
print("I'm running Python!")

I'm running Python!


Below we use a simple python function to convert Celsius degrees to Fahrenheit degrees.

In [0]:
%python

#convert celsius to fahrenheit
def celsiusToFahrenheit(source_temp=None):
    return(source_temp * (9.0/5.0)) + 32.0    
        
#input values - celsius
a = [1, 2, 3, 4, 5]
print(a)

#convert all
b = map(lambda x: celsiusToFahrenheit(x), a)
print(list(b))

[1, 2, 3, 4, 5]
[33.8, 35.6, 37.4, 39.2, 41.0]


##![Spark Logo Tiny](https://files.training.databricks.com/images/wiki-book/general/logo_spark_tiny.png) Databricks File System - DBFS

We've already imported data into Databricks by uploading our files.

Databricks is capable of mounting external/remote datasources as well.

DBFS allows you to mount storage objects so that you can seamlessly access data without requiring credentials.
Allows you to interact with object storage using directory and file semantics instead of storage URLs.
Persists files to object storage, so you won’t lose data after you terminate a cluster.

* DBFS is a layer over a cloud-based object store
* Files in DBFS are persisted to the object store
* The lifetime of files in the DBFS are **NOT** tied to the lifetime of our cluster

See also <a href="https://docs.azuredatabricks.net/user-guide/dbfs-databricks-file-system.html" target="_blank">Databricks File System - DBFS</a>.

### Databricks Utilities - dbutils
* You can access the DBFS through the Databricks Utilities class (and other file IO routines).
* An instance of DBUtils is already declared for us as `dbutils`.

The `mount` command allows to use remote storage as if it were a local folder available in the Databricks workspace

```
dbutils.fs.mount(
  source = f"wasbs://dev@{data_storage_account_name}.blob.core.windows.net",
  mount_point = data_mount_point,
  extra_configs = {f"fs.azure.account.key.{data_storage_account_name}.blob.core.windows.net": data_storage_account_key})
```

To show available DBFS mounts:

In [0]:
%fs 
mounts

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,sse-s3
/databricks/mlflow-tracking,databricks/mlflow-tracking,sse-s3
/databricks-results,databricks-results,sse-s3
/databricks/mlflow-registry,databricks/mlflow-registry,sse-s3
/,DatabricksRoot,sse-s3


To show available tables:

In [0]:
%fs
ls /FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/nyc_taxi.csv,nyc_taxi.csv,762417,1654895022000


Additional help is available via `dbutils.help()` and for each sub-utility: `dbutils.fs.help()`, `dbutils.meta.help()`, `dbutils.notebook.help()`, `dbutils.widgets.help()`.

See also <a href="https://docs.azuredatabricks.net/user-guide/dbutils.html" target="_blank">Databricks Utilities - dbutils</a>

In [0]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point updateMount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Similar to mount(), but updates an existing mount point (if present) instead of creating a new one fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory

In [0]:
df = spark.read.csv('/FileStore/tables/nyc_taxi.csv', header="true", inferSchema="true")

In [0]:
df.head()

Out[10]: Row(passengerCount=1.0, tripDistance=9.4, hour_of_day=15, day_of_week=2, month_num=1, normalizeHolidayName='None', isPaidTimeOff=False, snowDepth=29.058823529411764, precipTime=24.0, precipDepth=3.0, temperature=6.18571428571429, totalAmount=44.3)